In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
from pathlib import Path
if iskaggle: path = Path('/kaggle/input/digit-recognizer')
else:
    import zipfile,kaggle
    path = Path('.')
    paths = path/'digit-recognizer'
    if not paths.exists():
        ! kaggle competitions download -c digit-recognizer
        zipfile.ZipFile('digit-recognizer.zip').extractall(path)


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as img
from PIL import Image
import numpy as np
import tensorflow as tf 


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
train = tf.concat([x_train, x_test], 0)

In [ ]:
label = tf.concat([y_train , y_test ], 0 )

In [ ]:
plt.figure(figsize=(10,10))
for i in range(30):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train[i] , cmap=plt.cm.binary)
    plt.xlabel(int(label[i]))
plt.show()


In [ ]:
test = pd.read_csv(path/'test.csv')

In [ ]:
test = np.asarray(test)

In [ ]:
test = test.reshape(-1,28,28)

In [ ]:
print(test.shape)

In [ ]:
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test[i] , cmap=plt.cm.binary)
plt.show()


In [ ]:
train = train/255
test = test/255

In [ ]:

import keras
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.activations import relu, sigmoid


In [ ]:
inputs = Input(shape = (28 , 28, 1))
x = Conv2D(filters  = 8,kernel_size= 4, strides = 1, padding = 'SAME' )(inputs)
x = MaxPool2D(pool_size=8, strides=1, padding='SAME')(x)
x = Conv2D(filters  = 8,kernel_size= 16, strides = 2, padding = 'SAME',  activation= relu )(x)
x = MaxPool2D(pool_size=8, strides=2, padding='SAME')(x)
x = Conv2D(filters  = 8,kernel_size= 32, strides = 2, padding = 'SAME',  activation= relu )(x)
x = MaxPool2D(pool_size=8, strides=3, padding='SAME')(x)
x = Conv2D(filters  = 8,kernel_size= 28, strides = 1, padding = 'SAME',  activation= relu )(x)
x = MaxPool2D(pool_size=8, strides=3, padding='SAME')(x)
x = Flatten()(x)
x = Dense(256 , activation = relu)(x)
x = Dense(128 , activation = relu)(x)
x = Dense ( 128 , activation = relu)(x)
x = Dense ( 128 , activation = relu)(x)
x = Dense ( 128 , activation = relu)(x)
x = Dense(10 , activation = sigmoid)(x)
model =tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.summary()


In [ ]:
history = model.fit(train, label, batch_size=64, epochs=25, validation_split=0.2)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(25)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
ans =  pd.read_csv(path/'test.csv')

In [ ]:
ans = ans.drop(ans[:] , axis =1 )

In [ ]:
sol = model.predict(test)

In [ ]:
sol = sol.astype(float)
print(sol)

In [ ]:
sol = [np.argmax(i) for i in sol]

In [ ]:
print(sol)

In [ ]:
ans['label'] = sol 

In [ ]:
ans.reset_index(level=0, inplace=True)

In [ ]:
ans.rename(columns = {'index':'ImageId', 'label':'Label',
                              }, inplace = True)

In [ ]:
ans['ImageId'] += 1
print(ans)

In [ ]:
ans.to_csv('sol.csv', index = False)